
# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones
---

<p style="text-align: center;">
<img src=http://www2.famaf.unc.edu.ar/~efernandez/egeo/img/logos/famaf.jpg width=40%>
</p>

 Universidad Nacional de Córdoba

---

## Practico
### Aprendizaje No Supervisado

En la siguiente notebook se presentará la consigna a seguir para el cuarto y último práctico del proyecto, correspondiente a la materia Aprendizaje Automático No Supervisado. El objetivo consiste en aplicar distintas técnicas de análisis exploratorio de datos (EDA) al dataset.

> Usar el Dataset Original: [here](https://drive.google.com/file/d/1wCNrWQbUd_bh1mdjZ-dg3g9y0r02JyQN/view?usp=sharing)

## 1. Preprocesamiento
* Repetir el punto 2 del practico de Curación y Exploración de Datos
* Eliminar duplicados
* Normalizar los montos usando los indices de la inflacion
* Codificar usando hot encoding variables (Aquellas utilizadas para los practicos anteriores)

### Agrupamiento
Agrupar por mes en un ["balanced panel"](https://en.wikipedia.org/wiki/Panel_data#Example)
> Hint: [pandas.pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) 

In [1]:
import pandas as pd 
import numpy as np
import warnings

warnings.filterwarnings('ignore')

df = pd.read_parquet("cupones_ds.parquet")
df.drop_duplicates(keep="first", inplace=True, ignore_index=True)

In [2]:
cols = ['dni_titular_movimiento', 'monto_compra_movimiento', 'fecha_cupon_movimiento']
sample = df[cols].copy()

sample.loc[:, 'fecha_cupon_movimiento'] = pd.to_datetime(
    sample['fecha_cupon_movimiento'],  
    infer_datetime_format=True
)

sample['yr_month'] = sample['fecha_cupon_movimiento'].dt.strftime("%Y-%m")
sample

,dni_titular_movimiento,monto_compra_movimiento,fecha_cupon_movimiento,yr_month
0,0001686b52949b5461ffcbc766687e45031,6.24,2020-08-25,2020-08
1,000220fa96ec5af89817894033f8099c547,17.73,2020-08-25,2020-08
2,0002be202de47dfae9cc2304d91161be595,6.18,2020-08-25,2020-08
3,000e137d0af42e193be1ff670c00d4d1506,2.83,2020-08-25,2020-08
4,0009d010e4faf69552a814a33832b185877,2.38,2020-08-25,2020-08
...,...,...,...,...
124311,0001aa95aa3213844d85134123daadf0109,1200.00,2021-04-06,2021-04
124312,00061318fc7e5c1ab9d2ed1f02198877873,2244.42,2020-12-29,2020-12
124313,000c9a8174f608ccdbdf9d86b541c4af417,4480.00,2020-10-28,2020-10
124314,00089c2bc7f672981aaef9afd9a071c7911,3590.00,2021-02-13,2021-02


In [3]:
cols = ['dni_titular_movimiento', 'monto_compra_movimiento', 'yr_month']
pd.pivot_table(
    sample[cols], 
    values='monto_compra_movimiento', 
    index=['dni_titular_movimiento', 'yr_month'],
    aggfunc=np.sum,
    fill_value=0
).reset_index()

,dni_titular_movimiento,yr_month,monto_compra_movimiento
0,000000b5aea2c9ea7cc155f6ebcef97f826,2020-08,-10.68
1,000000b5aea2c9ea7cc155f6ebcef97f826,2020-09,-10.68
2,000000b5aea2c9ea7cc155f6ebcef97f826,2020-10,-10.68
3,000000b5aea2c9ea7cc155f6ebcef97f826,2020-11,-10.68
4,000000b5aea2c9ea7cc155f6ebcef97f826,2020-12,-10.68
...,...,...,...
7594,000f0b73ebfa002a79a0642b82e87919904,2021-01,30585.18
7595,000f0b73ebfa002a79a0642b82e87919904,2021-02,33979.54
7596,000f0b73ebfa002a79a0642b82e87919904,2021-03,7574.70
7597,000f0b73ebfa002a79a0642b82e87919904,2021-04,14997.16


### Creación de Nuevos Atributos
En esta etapa, pueden adicionar atributos calculados a partir de los datos preexistentes,
que crean que son pertinentes a priori o que hayan encontrado interesantes
* Crear una columna que indique si un cliente ha aumentado el monto de movimiento mesual, comparando con el mes anterior. En donde
  - 1 si el monto ha aumentado respecto del mes anterior (con un incremento mayor o igual al 10%)
  - 0 si no fue asi.
    > consideraciones: el primer mes del dataset, el valor es 0

* Convertir la variable `domicilio_codigo_postal` a continua 
  - Usando un codigo postal de referencia, crear un nuevo atributo con la distancia entre
    los distintitos codigos postales y el de referencia.

In [4]:
import pgeocode

dist = pgeocode.GeoDistance('ar')
# distancia aprox. entre jujuy y cordoba en Km
dist.query_postal_code("4600", "5000")

809.0582851446305

* Mean Encofing 
  - 1. Seleccionar una variable categórica que le gustaría transformar. 
  - 2. Agrupar por la variable categórica y obtener la suma agregada sobre la variable `"target"`.
  - 3. Agrupar por la variable categórica y obtener un recuento agregado sobre la variable `"target"` 
  - 4. Dividir los resultados del paso 2 / paso 3 y unir nuevamente con el dataset.

![image](https://miro.medium.com/max/700/1*iiM9g-qCa-Vff_HAFk-ppQ.png)


Usando la variable que indica si un cliente ha aumentado el monto de movimiento mesual como `target` y
como variable a codificar 
* cargo_descripcion_histo (variable recategorizada en el practico 2)
* nivel_estudio_descripcion_histo
* rel_vivienda_descripcion_histo
> conciderar a los Nulos como una categoria

Tabien codificar aquellas categoricas que concideren necesarias (del dataset original o del modificado).

### Normalización de Atributos
Es posible que sea necesario normalizar las features de nuestro dataset, dado que los algoritmos de clasificación no supervisada lo requieren. Aplicar al dataset la normalización de atributos que consideren adecuada.

### 2. Aplicación de Modelos de Aprendizaje Automático No Supervisado
Una vez finalizada la etapa de preprocesamiento, se propone implementar,
por un lado, una técnica de reducción de dimensionalidad para representar
los datos y, por el otro, dos técnicas de clusterización.

#### Descomposición de Variables: Principal Component Analysis
Se puede aplicarse PCA a las variables continuas
(edad, antiguedad, cantidad de compras por cada tipo de producto, etc.)

#### Visualizar los datos usando t-distributed Stochastic Neighbor Embedding.
* Se pueden sacar conclusiones del grafico?

#### K-Means Clustering
Explorar distintas soluciones de clustering con diferentes parámetros,
como iteraciones, número de clusters o métricas de distancia, y compararlas. 
Finalmente, para el modelo seleccionado:
> Aplicar K-Means tanto al dataset original como al dataset transformado.

* Calcular las métricas pertinentes sobre los clusters resultantes.
* Aplicando el método de Elbow, ¿cuál sería la cantidad óptima de clusters?
* Agregar al dataset el cluster resultante de los modelos.
* Graficar diferentes variables de interés por cluster y compararlos.

#### HDBScan Clustering.
Este algoritmo es de clusterización apropiado para detectar anomalias (outliers).
* comparar los clusters con los del punto anterior usando silhouette Analisys

#### K-Means Clustering para Serires Temporales
La idea es segmentar a los clientes en base a sus series temporales
usando el monto y la fecha del monto de moviento.
* Usar la libreria [ts-learn](https://tslearn.readthedocs.io/en/stable/auto_examples/clustering/plot_kmeans.html) 


> Agregar al dataset el cluster resultante de los modelos.

### 3. Optativo: Clustering Agglomerativos
Usar cluster aglomerativos para hacer clusterning de series temporales
usando el monto y la fecha del monto de moviento.
*  [hierarchy](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html)